In [21]:
import numpy as np
import pandas as pd
import os

### 切換路徑

In [22]:
os.getcwd()

'/home/Oring-AI/Model_II'

In [23]:
os.chdir("/home/Oring-AI/Model_II")
os.getcwd()

'/home/Oring-AI/Model_II'

In [24]:
### 讀取資料 - dimension stone & sheet size
detail = pd.read_csv("Demand.csv")

### Dimension_stone

In [ ]:
import re
pattern=r'0{5}$'
dimension_stone = detail[~detail["Number"].str.contains(pattern)]
dimension_stone.reset_index(drop=True, inplace=True)

In [ ]:
dimension_stone.groupby("Number").size().sort_values(ascending = False).to_csv("material_by_number_daily.csv")

In [ ]:
material_by_number = dimension_stone.groupby("Number").size().sort_values(ascending = False).to_frame(name="Count")
material_by_number.reset_index(inplace=True)

### Transform the data to weekly data

In [ ]:
import datetime
 
def dateRange(beginDate, endDate):
    dates = []
    date = beginDate
    
    while date <= endDate:
        dates.append(date)
        date = date + datetime.timedelta(1)
    
    return dates

In [ ]:
start = datetime.datetime.strptime('2013/05/01', "%Y/%m/%d")
end = datetime.datetime.strptime('2020/04/30', "%Y/%m/%d")

dates = dateRange(start, end)
dates_data = pd.DataFrame(dates, columns=["Date"])

In [ ]:
dimension_stone["Date"] = pd.to_datetime(dimension_stone['Date'], format='%Y/%m/%d')

In [ ]:
Numbers = dimension_stone["Number"].unique()

In [ ]:
def dataToWeek(data, step= 7):
    
    weekly_df = pd.DataFrame()
    
    weekly_df = weekly_df.append([np.round(np.mean(data.iloc[i:i+step]),2) for i in range(0, len(data), step)])

    return weekly_df

### Transform daily data to weekly data

In [ ]:
material_weekly = pd.DataFrame(columns=["Number", "Count"])

for Number in Numbers:
    material = dimension_stone[dimension_stone["Number"]==Number]

    data = pd.merge(dates_data, material, how='left', on='Date')
    data = data["Amount"]
    weekly_data = dataToWeek(data, step= 7)
    
    new = {
        "Number" : material["Number"].values[0],
        "Count" : int(weekly_data.count())
    }

    material_weekly = material_weekly.append(new,ignore_index=True) 
        ##Way I
#         print (sum(np.isnan(weekly_data)==False))

        ##Way II
#         amount = Data["Amount"]
#         print (len(amount[~np.isnan(amount)]))

In [ ]:
material_weekly.sort_values(by = 'Count', ascending=False, ignore_index=True).to_csv("material_by_number_weekly.csv", index=False)